<a href="https://colab.research.google.com/github/bernlkb/STQD6014-DataScience/blob/main/Week14_Modelling_20240121_WC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Week 14: Introduction to Modelling**

Please refer to the textbook: "Python for Data Analysis" by Wes McKinney for details of this topic on Chapter 12




In [1]:
# start by importing NumPy and pandas
import numpy as np
import pandas as pd

### **12.1 Interfacing Between pandas and Model Code**

* The point of contact ***between pandas and other analysis libraries*** is usually ***NumPy arrays***

***Turn a DataFrame into a NumPy array***

In [2]:
# a Pandas DataFrame
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
# Get the data column names
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

### The ***to_numpy()*** method is intended to be used

* when our data is ***homogeneous***—for example, all numeric types.

In [4]:
# Turn the pandas DataFrame into NumPy array
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
# Convert the numpy array back to DataFrame
df2 = pd.DataFrame(data.to_numpy(), columns = ['Hasanah', 'Suraya', 'Azlin'])
df2

,Hasanah,Suraya,Azlin
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
# Original form of data
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [8]:
# Extract subset of the columns - using loc method
# loc method -> use the label explicitly
# get numpy array
model_cols = ['x0', 'x1']
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [10]:
# Add a column with categorical value
# Could be important for final exam -> adding new column
data['category'] = pd.Categorical(['a', 'b', 'a', 'b', 'b'],
                                  categories = ['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,b
4,5,0.00,-2.0,b


In [11]:
# Create dummy dataframe based on categorical value
# focusing on the 'category' variable
# one-hot encoding
dummies = pd.get_dummies(data.category, prefix='cat')
dummies

,cat_a,cat_b
0,1,0
1,0,1
2,1,0
3,0,1
4,0,1


In [12]:
# Display original data content
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,b
4,5,0.00,-2.0,b


In [13]:
# Convert and join with transformed categorical dataset
data_w_dummies = data.drop('category', axis = 1).join(dummies)
data_w_dummies

,x0,x1,y,cat_a,cat_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,0,1
4,5,0.00,-2.0,0,1


### **12.2 Creating Model Descriptions with Patsy**


1.   Patsy -> Python library for describing statistical models (especially linear models)
2.   special string syntax: ***y ~ x0 + x1***
3. describe the relationship between a ***response variable (y)*** and ***one or more covariates or features (in this case, x0 and x1)***.
    * the ***variable y*** is ***modeled by a linear combination*** of the ***variables x0 and x1***.
    * e.g. ***Weight*** is modeled by a ***linear combination*** of the ***Age and Gender***




In [14]:
# import the patsy library
import patsy

In [15]:
# example pandas dataframe
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [16]:
# Creating the model using dmatrices() function -> design matrix
# a + b does not mean adding a to b
# these are terms in the design matrix created for the model
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [17]:
# Getting y design matrix of size 5 x 1
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [18]:
# Getting X design matrix of size 5 x 3
# patsy automatically adds the intercept term = 1 to facilitate linear modeling
# doesn't imply that the intercept is 1; it's just a placeholder in the design matrix
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [19]:
# Convert y to numpy array
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [20]:
# convert X to numpy array
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

# ***Intercept term***
* intercept term represents the ***value of the response variable*** when all ***covariates (x0, x1, ...) are zero***
* In certain situation, ***omitting the intercept term might make sense***
    * Imagine we are studying a chemical reaction, and we are trying to **model the yield of a product (the response, *y*)** based on the amounts of two reactants, A and B ***(the covariates, x0, x1)***
        * ***a reaction with zero amounts of both A and B might reasonably be expected to produce zero yield.***

In [26]:
# Suppress the intercept term (if we want)
patsy.dmatrices('y ~ x0 + x1 + 0', data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

In [27]:
# Perform ordinary least squares regression using numpy.linalg.lstsq
# NumPy Linear Algebra Least Squares
# rcond=None -> letting the function automatically handle the sensitivity or stability of the matrix computations
# reciprocal condition number is used to express how well-conditioned a problem is, where a higher value indicates better conditioning
# "_" -> used as a convention to represent variables that we're not explicitly interested in or not using
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond = None)

In [28]:
# Output four elements
np.linalg.lstsq(X, y, rcond = None)

(array([[ 0.31290976],
        [-0.07910564],
        [-0.26546384]]),
 array([19.63791494]),
 3,
 array([8.03737688, 3.38335321, 0.90895207]))

In [29]:
# Get the coefficient value
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [31]:
# reattach the model column names to the fitted coefficients to obtain a meaningful Series
# associating each coefficient with its corresponding variable name
coef = pd.Series(coef.squeeze(), index = X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

# **Model formula**

* $y = B_0 + x_0X_0 + x_1X_1$
* $y = 0.312910 - 0.079106X_0 - 0.265464X_1$

# ***Residual***
1. residuals are the ***differences between the actual observed values*** of the ***response variable*** and the ***values predicted by the linear model***
2. Each data point has a residual, which is the ***vertical distance*** ***between the observed point and the point predicted by the model***.
3. ***Small Residuals***
    * When the ***sum of squared residuals is small***, it suggests that the ***model does a good job of explaining the variation in the data***
    * the predicted values are very close to the actual observed values
    * A smaller sum of squared residuals indicates a better fit of the model to the data
4. ***Large Residuals***
    * a l***arge sum of squared residuals suggests that the model does not fit the data well***
    * The predicted values deviate significantly from the actual observed values
    * Larger residuals indicate that the ***model is not capturing some of the variability*** in the data, and there may be room for improvement.

In [32]:
# Residual value
pd.Series(resid, name = 'Residuals')

0    19.637915
Name: Residuals, dtype: float64

In [33]:
# model metadata is retained in the design_info attribute
X.design_info

DesignInfo(['Intercept', 'x0', 'x1'],
           factor_infos={EvalFactor('x0'): FactorInfo(factor=EvalFactor('x0'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('x1'): FactorInfo(factor=EvalFactor('x1'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('x0')]),
                                      [SubtermInfo(factors=(EvalFactor('x0'),),
                                                   contrast_matrices={},
     

#### **Data Transformations in Patsy Formulas**

In [34]:
# Display original form of data
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [35]:
# Mixing Python code into Patsy formulas
# transform x1
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [36]:
# Original content for data
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [37]:
# Getting mean for x0
np.mean(data['x0'])

3.0

In [38]:
# Getting standard deviation
np.std(data['x0'])

1.4142135623730951

In [39]:
# Getting mean for x1
np.mean(data['x1'])

-0.7699999999999999

In [41]:
# Variable transformations
# standardize function standardizes the variable by subtracting its mean and dividing by its standard deviation
# rescales the variable x0 to have a mean of 0 and a standard deviation of 1
# center -> minus the variable x1 from its mean
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [42]:
# How to get standardized value
mean1 = np.mean(data.x0)
std2 = np.std(data.x0)
(2 - mean1) / std2

-0.7071067811865475

In [43]:
# How to get centering value
-0.01 - np.mean(data.x1)

0.7599999999999999

In [44]:
# Create a new pandas dataframe
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [45]:
# fitting a model on one dataset, then evaluate the model based on another dataset
# transformations to new out-of-sample data
# [X.design_info] is a list containing the design information from the "previous design matrix X"
# patsy.build_design_matrices -> is used to build design matrices for new data based on the design information from the original data
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [46]:
# Meta data from the previous design matrix X
X.design_info

DesignInfo(['Intercept', 'standardize(x0)', 'center(x1)'],
           factor_infos={EvalFactor('standardize(x0)'): FactorInfo(factor=EvalFactor('standardize(x0)'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('center(x1)'): FactorInfo(factor=EvalFactor('center(x1)'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('standardize(x0)')]),
                                      [SubtermInfo(factors=(EvalFactor('standardiz

In [ ]:
#DesignInfo(['Intercept', 'standardize(x0)', 'center(x1)'], # -> indicates the terms included in the design matrix
#           factor_infos={EvalFactor('standardize(x0)'): FactorInfo(factor=EvalFactor('standardize(x0)'), # -> provides information about each factor (variable) in the design matrix
#                                    type='numerical',
#                                    state=<factor state>,
#                                    num_columns=1),
#                         EvalFactor('center(x1)'): FactorInfo(factor=EvalFactor('center(x1)'),
#                                    type='numerical',
#                                    state=<factor state>,
#                                    num_columns=1)},
#           term_codings=OrderedDict([(Term([]), # -> describes how each term in the design matrix is coded; Term([]) -> Intercept
#                                      [SubtermInfo(factors=(),
#                                                   contrast_matrices={},
#                                                   num_columns=1)]),
#                                     (Term([EvalFactor('standardize(x0)')]),
#                                      [SubtermInfo(factors=(EvalFactor('standardize(x0)'),),
#                                                   contrast_matrices={},
#                                                   num_columns=1)]),
#                                     (Term([EvalFactor('center(x1)')]),
#                                      [SubtermInfo(factors=(EvalFactor('center(x1)'),),
#                                                   contrast_matrices={},
#                                                   num_columns=1)])]))

In [47]:
# Orignal content of data
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [48]:
# By default, the plus symbol (+) in the context of Patsy formulas does not mean addition
# The I() function -> to explicitly specify that the response variable y is related to the "sum of x0 and x1"
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

#### **Categorical Data and Patsy**

1.   When using ***nonnumeric terms*** (***Categorical Data***) in a Patsy formula, they are converted to dummy variables by default



In [49]:
# Example dataframe
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [51]:
# building model with categorical data
# one of the levels will be left out to avoid collinearity
# "[T.b]" -> 'treatment b' -> refers to level "b" of the variable
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [52]:
# Omit intercept
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [53]:
# Numeric columns can be interpreted as categorical with the C function
# C() function -> indicate that a variable should be treated as categorical
# [T.1] -> refers to Treatment 1
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [54]:
# Display original content of data
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [55]:
# Data with multiple categorical terms
data['key2'] = data['key2'].map({0: 'zero', 1:'satu'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,satu,2,0.0
2,b,zero,3,2.5
3,b,satu,4,-0.5
4,a,zero,5,4.0
5,b,satu,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [56]:
# Creating the model
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [57]:
# Original content of data
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,satu,2,0.0
2,b,zero,3,2.5
3,b,satu,4,-0.5
4,a,zero,5,4.0
5,b,satu,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [58]:
# Another example with multiple categorical terms
# 'key1:key2' -> interaction between key1 and key2
# allows the model to capture the combined effect of both key1 and key2 on the response variable y
# cannot explained by considering key1 and key2 separately
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# **12.3 Introduction to statsmodels**
statsmodels is a Python library for ***fitting*** many kinds of ***statistical models***, performing statistical tests, and data exploration and visualization.

#### **Estimating Linear Models**

In [ ]:
# load statsmodels module
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Generate a linear model using some random data
# dnorm generate normally distributed data with a particular mean and variance
rng = np.random.default_rng(seed=12345)

# Define a dnorm function
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

# Create the data
# "np.c_" -> concatenate arrays along the second axis; stacks the arrays horizontally, column-wise.
N=100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

# Get the y values
y = np.dot(X, beta) + eps

In [ ]:
# Extract eps values
eps[:3]

array([ 0.0612879 , -0.04088595,  0.11209613])

In [ ]:
# Extract X values
X[:3]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899]])

In [ ]:
# Extract y values
y[:3]

array([-0.59952668, -0.58845445,  0.18563386])

In [ ]:
# Manual calculations
# eps: 0.0612879 , -0.04088595,  0.11209613
# beta = [0.1, 0.3, 0.5]
# y = np.dot(X, beta) + eps
((-0.90050602 * 0.1) + (-0.18942958 * 0.3) + (-1.0278702 * 0.5)) + 0.0612879

-0.599526676

In [ ]:
# sm.add_constant() function -> add an intercept column to an existing matrix


In [ ]:
# Extract X_model data


array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899]])

In [ ]:
# What is X
X

array([[-9.00506021e-01, -1.89429577e-01, -1.02787020e+00],
       [ 7.99252054e-01, -1.54598388e+00, -3.27397080e-01],
       [-5.50654833e-01, -1.20254287e-01,  3.29358994e-01],
       [-1.63915546e-01,  8.24039852e-01,  2.08274848e-01],
       [-4.76512913e-02, -2.13146980e-01, -4.82436357e-02],
       [-4.68576597e-01, -1.43558784e+00, -1.52694953e-01],
       [-8.65068061e-01, -9.63148432e-02,  7.08625055e-01],
       [ 4.10395842e-01,  6.08038650e-01,  1.26222105e-01],
       [ 2.28353201e-01,  1.56467440e-01,  4.06761512e-01],
       [-1.23509905e+00, -3.31585038e-01,  1.76681376e-01],
       [ 1.48463222e+00,  1.43167842e+00, -2.99354280e-01],
       [ 6.12531226e-01,  1.47169718e+00,  6.95582154e-01],
       [-4.80278623e-01, -7.62397041e-02, -5.53712608e-01],
       [ 5.70600290e-01,  6.30092128e-01, -5.34945034e-01],
       [-2.95327118e-01,  3.04024846e-01, -1.91921494e-01],
       [-3.83834219e-02,  6.05303068e-01, -3.26313824e-01],
       [ 4.98908970e-01,  1.12569928e+00

In [ ]:
# What is y
y

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445,
       -0.48405182,  0.03006418,  0.21745535,  0.09733398,  0.29428061,
       -0.55821726,  0.39275937, -0.88717278, -0.14095687, -0.24884462,
       -0.11545172,  0.49031861, -0.53928623,  0.01003077, -0.12181392,
       -0.40652191, -0.26296953,  0.24121365, -0.01486882, -0.8269326 ,
        0.85796862, -0.1581605 ,  0.3229089 , -0.3182448 , -0.25177682,
        0.01201277, -0.27692688,  0.48915537,  0.02713607,  0.32623478,
       -0.67005187, -0.43638036,  0.19876113,  0.29108143,  1.22925781,
       -0.13454699,  0.11618133, -0.28334439,  0.82639714,  0.65173309,
        0.36932726,  0.46060306, -0.36001815, -0.67943794, -0.32391053,
        0.22890353,  0.33392927, -0.02893472,  0.3514912 ,  0.4104761 ,
        0.02342595, -0.08816253, -0.42223253,  0.95031167, -0.84319881,
       -0.17742481, -0.58276729, -0.04786858,  0.49981461, -0.40998529,
       -0.06505591, -0.11920059, -0.73782536,  0.11294051, -0.50

In [ ]:
# sm.OLS() -> Create an ordinary least squares linear regression model
# predict the variable y using the information in the matrix X
# model -> this is generated using training data


# ***The fit() method***
1. **Creating a Model:**
    
    * Before using the fit() method, we create a model object.
    
2. **Training the Model:**
    
    * Once we have the model, it needs to learn from the data.
    * ***model.fit()*** -> we're telling the model to "learn" from the data we provide.

3. **Providing Data:**

    * The ***fit() method*** takes our input data, which includes both the ***features (covariates)*** and the ***target variable (response variable)***.
    * The model uses these data to ***adjust its internal parameters or coefficients***.

4. **Adjusting Parameters:**

    * During the training process, the model adjusts its parameters or coefficients to ***minimize the difference*** between its predictions and the actual target values in the training data.

5. **Resulting Trained Model:**

    * After the fit() method completes, we have a ***trained model***.
    * The model has ***learned patterns or relationships in the training data*** and is ready to make predictions or provide insights.

In [ ]:
# The model’s fit() method returns a regression results object
# containing estimated model parameters and other diagnostics
# params -> get the coefficient data


array([0.06681503, 0.26803235, 0.45052319])

In [ ]:
# summary() method on results can print a model detailing
# diagnostic output of the model


                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Sat, 20 Jan 2024   Prob (F-statistic):                    2.66e-13
Time:                        22:07:53   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

# **The model formula (based on the above OLS output):**
# General formula:
# **$y = β_0 + β_1x_0 + β_2x_1 + β_3x_2 + ϵ$**
# **$y = 0.0668x_0 + 0.2680x_1 + 0.4505x_2 + ϵ$**

# ***Digest the OLS output***

# ***Model Fit:***
1. ***R-squared (uncentered): 0.469*** -> proportion of the variance in the response **variable** explained by the covariates; 46.9% of the variance is explained.
2. ***Adjusted R-squared (uncentered)***: Similar to R-squared but ***adjusted for the number of covariates***; 45.2% in this case

# ***Model Characteristics:***
1. ***F-statistic:*** A measure of overall fit (looking at individual effects)
    
    * **Purpose**: The F-statistic tests whether the overall model (all the covariates together) is statistically significant.
    * **Interpretation**: A high F-statistic suggests that at least one covariate is contributing significantly to explaining the variability in the response variable.
    * **Real-world analogy**: Imagine we are trying to predict a student's final exam score based on **hours spent studying**, **attendance**, and **sleep hours**.
    * The F-statistic helps us determine if, overall, these factors combined have a significant impact on the final exam scores.
    * ***Higher values indicate a better fit***

2. ***Prob (F-statistic):*** The probability associated with the F-statistic.
    * ***A low value suggests a strong evidence against null hypothesis.***

    * ***Null Hypothesis (H0)***: all the coefficients in the model are equal to zero, meaning ***none of the covariates have a significant effect on the response variable.***
    * ***Alternative Hypothesis (H1):*** at least one of the coefficients is not equal to zero, indicating that ***at least one covariate has a significant effect on the response variable***.
    * The ***F-statistic*** is then used to ***test whether the improvement in model fit (explained by the regression)*** is ***statistically significant compared to a model with no covariates (only the intercept)***

# ***Coefficients:***
1. ***std err:*** a measure of the ***variability or precision associated with each coefficient (slope) in the model***.

    * A ***low standard error*** suggests that the estimate of the coefficient is likely to be very close to the true population value
    * A ***high standard error***, on the other hand, indicates that there is more uncertainty or variability in the estimate.
2. ***t:*** The t-statistic, indicating ***how many standard errors*** the coefficient is ***away from zero.***
    * ***large t-statistic*** suggests that the corresponding coefficient is likely to be statistically different from zero.
    * ***The sign of the t-statistic*** indicates the ***direction of the relationship (positive or negative)*** between the covariates and the response variable.
3. ***P>|t|:*** The p-value associated with the t-statistic.
    * ***Low p-values indicate high confidence for alternative hypothesis***
    * ***H0:*** The null hypothesis states that the specific coefficient associated with a covariate is equal to zero, meaning that ***the variable has no significant effect on the response variable.***
    * ***H1:*** The alternative hypothesis asserts that the specific coefficient is not equal to zero, indicating that ***the variable has a significant effect on the response variable.***
4. **[0.025 0.975]:** The ***95% confidence interval*** for the coefficients.

# ***Model Diagnostics:***
1. ***Omnibus***
    * assesses the ***overall model fit*** by examining whether the regression model explains a significant amount of variance in the ***response variable***.
    * ***High Value***: suggests that the model is doing a good job of explaining the variation in the response variable
    * **Real-world analogy**: Think of the Omnibus test as a comprehensive evaluation of our model's ability to predict.
        * Going back to the student example, it assesses how well our combined predictors (study hours, attendance, sleep) collectively explain the variations in final exam scores.

2. ***Durbin-Watson***
    * detect the presence of autocorrelation
    * The Durbin-Watson statistic is a number that ***falls between 0 and 4***.
    * ***Close to 2***: A value around 2 suggests that there is ***no significant autocorrelation in the residuals (good).***
    * Significantly Below 2: Indicates positive autocorrelation.
    * Significantly Above 2: Indicates negative autocorrelation.
3. ***Jarque-Bera***
    * assesses whether the residuals of a regression model have a normal distribution.
    * High Value: A high Jarque-Bera value suggests that the residuals may not be normally distributed.
    * ***Low Value***: A low Jarque-Bera value indicates that the residuals are more likely to be ***normally distributed.***

4. ***Skew***
    * measures the asymmetry of the distribution of residuals.
    * Positive Skew: Residuals are skewed to the right (long tail on the right).
    * Negative Skew: Residuals are skewed to the left (long tail on the left).
    * ***Skew Close to 0: Residuals are approximately symmetric.***
5. ***Kurtosis***
    * measures the "tailedness" of the distribution of residuals.
    * High Kurtosis: Residuals have heavy tails, indicating more extreme values.
    * ***Low Kurtosis: Residuals have lighter tails, indicating a more normal distribution.***

# ***Information Criteria:***
1. ***AIC (Akaike Information Criterion)***
    * A measure of the model's goodness of fit, adjusted for the number of covariates.
    * ***balances the trade-off between the complexity of the model (number of parameters)*** and its ability to ***explain the observed data***.
    * ***Lower AIC***: A lower AIC value is generally preferred. It suggests that the model ***provides a good fit to the data while penalizing for model complexity***.
    * When comparing models, a model with a ***lower AIC is considered to be more parsimonious*** (simpler) while still providing an adequate fit to the data.

2. ***BIC (Bayesian Information Criterion)***
    * Similar to AIC but penalizes model complexity more.
    * BIC tends to ***favor simpler models more than AIC***, especially in situations where the sample size is small. It penalizes additional parameters more severely.

3. ***Low Values (AIC and BIC)***:

    * ***Low AIC and BIC values*** indicate that the model is ***both effective in explaining the data and relatively simple.***

In [ ]:
# Put the model parameters in a DataFrame
data = pd.DataFrame(X, columns = ['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [ ]:
# fitting the model


In [ ]:
# Display the OLS result summary


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     28.36
Date:                Sat, 20 Jan 2024   Prob (F-statistic):           3.23e-13
Time:                        22:49:41   Log-Likelihood:                -25.390
No. Observations:                 100   AIC:                             58.78
Df Residuals:                      96   BIC:                             69.20
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0208      0.032     -0.653      0.5

# **The model formula (based on the above OLS output):**
# General formula:
# **$y =β_0 + β_1x_1 + β_2x_2 + β_3x_3 + ϵ$**
# **$y = -0.0208 + 0.0658col_0 + 0.2690col_1 + 0.4494col_2 + ϵ$**

# ***Key differences: sm.OLS() vs smf.ols()***
1. ***sm.OLS(y, X)*** requires us to pass the covariate and response variables directly as arrays or matrices.
    * need to have our data organized in a ***matrix (NumPy array or Pandas DataFrame)*** for both response and covariates.

2. ***smf.ols('y ~ col0 + col1 + col2', data=data)*** uses a ***formula approach***, allowing us to ***specify the model in a more concise and formulaic manner***.
3. Both methods achieve the same goal of fitting an OLS regression model

In [ ]:
# Extract the estimated coefficients (parameters) of the fitted model


Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [ ]:
# Extract the t-statistics associated with each estimated coefficient of the fitted model


Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

In [ ]:
# Original content of data
data

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374
...,...,...,...,...
95,-0.039152,0.531515,-0.587640,-0.067934
96,-0.227355,0.941139,-0.228237,0.831554
97,-0.473484,0.167359,-0.044659,0.070316
98,-0.610622,-0.747349,-0.057917,-0.386481


In [ ]:
# Compute predicted values
# Using test data [unseen data]


0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
5   -0.506392
6    0.214832
7    0.226481
8    0.219121
9   -0.111867
dtype: float64

# **12.4 Introduction to scikit-learn**
most widely used and trusted general-purpose Python machine learning toolkits.

In [ ]:
# Passenger survival rates on the Titanic in 1912
# Pclass - Passenger Class
# Sibsp - Number of Siblings/Spouses Aboard
# Parch - Number of Parents/Children Aboard
# Embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
# More information: http://tinyurl.com/ycp9t56s
train = pd.read_csv('https://bit.ly/3WbFzA9')
test = pd.read_csv('https://bit.ly/3QMh2kn')
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [ ]:
# checking for missing data in training dataset


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# checking for missing data in testing dataset


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
# use the median of the training dataset to fill the nulls in both tables


In [ ]:
# checking for missing data in transformed training dataset


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# add a column IsFemale as an encoded version of the 'Sex' column


In [ ]:
# Display top few rows of train data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [ ]:
# shortlisting some model variables


In [ ]:
# Display top few rows of X_train data
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [ ]:
# Display top few rows of y_train data
y_train[:5]

array([0, 1, 1, 1, 0])

In [ ]:
# create a Logistic Regression model instance
from sklearn.linear_model import LogisticRegression


In [ ]:
# fit the model to the training data using the model’s fit method


LogisticRegression()

In [ ]:
# Get the coefficient


array([[-1.14125712,  2.51957422, -0.03271783]])

In [ ]:
# Get the intercept


array([2.06560016])

# **The logistic regression model formula**:
# General formula: $logit(p) = β_0 + β_1x_1 + β_2x_2+ … +β_nx_n​$

# Our formula:
# $logit(p) = 2.066 - 1.141x_1 + 2.520x_2 - 0.033x_3$

# $x_1$ = 'Pclass', $x_2$ = 'isFemale', $x_3$ = 'Age'

# **To get predicted probability:**
# **p = 1 / (1 + exp(-logit(p)))**

In [ ]:
# Predict using the test dataset


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [ ]:
# Display original test dataframe
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isFemale
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1


In [ ]:
# Add the y_predict column into the test data


In [ ]:
# Top 20 rows
test[:20]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isFemale,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,1
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,0,0
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,1,1
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S,0,0
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,1,1
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S,0,0


In [ ]:
# another example using cross validation
# to avoid overfitting
# 'Cs = 10' indicate regularization parameter -> a technique used to prevent overfitting in a model
# A high regularization parameter encourages a simpler model
# A low regularization parameter allows the model to be more complex, potentially fitting the training data too closely.
from sklearn.linear_model import LogisticRegressionCV


LogisticRegressionCV()

In [ ]:
# Predict using the test dataset


with cross validation: [0 0 0 0 1 0 1 0 1 0]
no cross validation  : [0 0 0 0 1 0 1 0 1 0]


# **That's all for the day!!!**
# **See you all next week**